# test the bridge code

In [ ]:
%matplotlib inline

from binomial_bridge import BinomialBridge
from beta_bridge import BetaBridge

import numpy as np
import scipy
import sobol_seq
import matplotlib.pyplot as plt

#jump_times_list is a list of jump time paths, where each element corresponds to a path
def plot_jump_times_stats(jump_times_list, T, is_beta_bridge):
    times_interp = np.linspace(0, T, 100+1)

    if is_beta_bridge:
        #need to create a different set of jump times, otherwise the interarrival times become biased
        #by filling in the rest of the jump times between jump_times[-1] and T
        #however, by doing so we get the right mean and expectation of the counting process
        jump_times_interp = []
        for jump_times in jump_times_list:
            if jump_times[-1] >= T:
                jump_times_interp.append(jump_times)
            else:
                dt = T - jump_times[-1]
                y = np.random.poisson(dt)
                if y > 0:
                    new_times = jump_times[-1] + np.sort(np.random.uniform(0, dt, size=y))
                    jump_times_interp.append(np.concatenate((jump_times, new_times)))
                else:
                    jump_times_interp.append(jump_times)
    else:
        jump_times_interp = jump_times_list
        
    counts = [range(1, len(jump_times)+1) for jump_times in jump_times_interp]  
    
    interp_functions = [scipy.interpolate.interp1d(jump_times, count, kind='previous', fill_value=(0, count[-1]), bounds_error=False)
                        if jump_times.size > 1 else lambda x: np.zeros(x.size) for jump_times,count in zip(jump_times_interp, counts)]

    interp_counts = [func(times_interp) for func in interp_functions]
    interp_counts = np.vstack(interp_counts)

    plt.figure(figsize=(6,6))
    plt.plot(times_interp, times_interp, label='theoretical mean')
    plt.plot(times_interp, np.mean(interp_counts, axis=0), label='empirical mean')
    plt.legend()
    plt.xlabel('t')
    plt.ylabel('N(t)')
    plt.show()

    plt.figure(figsize=(6,6))
    plt.plot(times_interp, times_interp, label='theoretical variance')
    plt.plot(times_interp, np.var(interp_counts, axis=0), label='empirical variance')
    plt.legend()
    plt.xlabel('t')
    plt.ylabel('N(t)')
    plt.show()

    plt.figure(figsize=(6,6))
    interarrival_times = [np.diff(jump_times, prepend=0) for jump_times in jump_times_list]
    interarrival_times = np.concatenate(interarrival_times)

    quantiles, line = scipy.stats.probplot(interarrival_times, dist='expon', plot=plt)
    print('(slope, intercept, r): {}'.format(line))
    x = np.linspace(0, quantiles[0][-1], 101)
    plt.plot(x,x, color='green')
    plt.show()

################################
# parameters for the tests
################################
n = 10**6
discretization_power = 3
dim = 2**discretization_power
    
T = Y_at_T = 10

# points = sobol_seq.i4_sobol_generate(dim, n).T

points = np.random.uniform(size=(dim, n))

################################
# test the binomial bridge
################################
binom_bridge = BinomialBridge(T, discretization_power, points)
binom_bridge.generate()
binom_bridge.fill_jump_times()

plot_jump_times_stats(binom_bridge.jump_times, T, False)

# ################################
# # test the beta bridge
# ################################
beta_bridge = BetaBridge(Y_at_T, discretization_power, points)
beta_bridge.generate()
beta_bridge.fill_jump_times()

plot_jump_times_stats(beta_bridge.jump_times, Y_at_T, True)

# Y_at_T = 10000
# gamma = np.random.gamma(Y_at_T, 1, size=10)
# unifs = np.random.uniform(0, gamma, size=(Y_at_T-1, len(gamma)))
# sorted_unifs = np.sort(unifs, axis=0)
# jump_times = np.vstack((sorted_unifs, gamma))
# interarrivals = np.diff(sorted_unifs, prepend=0, axis=0)
# interarrivals = np.ravel(interarrivals)

# quantiles, line = scipy.stats.probplot(interarrivals, dist='expon', plot=plt)
# x = np.linspace(0, quantiles[0][-1], 101)
# plt.plot(x,x, color='green')
# plt.show()
# print('(slope, intercept, r): {}'.format(line))

In [23]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from tqdm import tnrange, tqdm_notebook

from binomial_bridge import BinomialBridge
from beta_bridge import BetaBridge

import numpy as np
import sobol_seq
import matplotlib.pyplot as plt

from model_data import ModelData
from birth_death_model import BirthDeathModel
from birth_model import BirthModel

from nrm_with_bridge import nrm_with_bridge
from gillespie import gillespie

# np.random.seed(0)

model_data = ModelData(BirthDeathModel(),
                       ['A'],
                       0.25,
                       'Birth-Death')
x0 = np.array([100])

# model_data = ModelData(BirthModel(),
#                        ['A'],
#                        2.,
#                        'Birth')
# x0 = np.array([10])

test_n = 10**6
n = 10**6

all_paths_states = np.zeros((test_n, len(x0)), dtype=int)
all_paths_num_jumps = np.zeros((test_n, model_data.model.R), dtype=int)
for i in tnrange(test_n, desc='Running test simulations'):
    _, state_path, num_jumps = gillespie(model_data.model, x0, model_data.t)
    all_paths_states[i,:] = state_path[-1,:]
    all_paths_num_jumps[i,:] = num_jumps
    
mean_jumps = np.mean(all_paths_num_jumps, axis=0)

print('mean: {}'.format(np.mean(all_paths_states, axis=0)))
print('var: {}'.format(np.var(all_paths_states, axis=0)))
print('mean number of jumps: {}'.format(mean_jumps))

##############################

discretization_power = 1
dim = 2**discretization_power

bridges = [BinomialBridge(T, discretization_power, np.random.uniform(size=(dim, n))) for T in mean_jumps]
for bridge in bridges:
    bridge.generate()
    bridge.fill_jump_times()

all_paths_states = np.zeros((n, len(x0)), dtype=int)
all_paths_num_jumps = np.zeros((n, model_data.model.R), dtype=int)
for i in tnrange(n, desc='Running bridge simulations'):
    jump_times = [bridge.jump_times[i] for bridge in bridges]
    _, state_path, num_jumps = nrm_with_bridge(model_data.model, x0, model_data.t, jump_times)
    all_paths_states[i,:] = state_path[-1,:]
    all_paths_num_jumps[i,:] = num_jumps

mean_jumps = np.mean(all_paths_num_jumps, axis=0)

print('mean: {}'.format(np.mean(all_paths_states, axis=0)))
print('var: {}'.format(np.var(all_paths_states, axis=0)))
print('mean number of jumps: {}'.format(mean_jumps))


mean: [88.945948]
var: [28.31135438]
mean number of jumps: [12.50434  23.558392]



mean: [88.939309]
var: [28.2693456]
mean number of jumps: [12.500589 23.56128 ]
